In [1]:
"""
Script to fine-tune BART for conversation prediction (partial -> remainder).

Differences from previous approach:
- Uses a simpler Hugging Face Dataset creation approach with map() for tokenization
- Minimal usage of DataCollatorForSeq2Seq
- Same logic of partial->remainder structure
"""


'\nScript to fine-tune BART for conversation prediction (partial -> remainder).\n\nDifferences from previous approach:\n- Uses a simpler Hugging Face Dataset creation approach with map() for tokenization\n- Minimal usage of DataCollatorForSeq2Seq\n- Same logic of partial->remainder structure\n'

In [2]:
# ==========================================
# 1) Imports
# ==========================================
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split

from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments
)
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



Using device: cpu


In [3]:
# ==========================================
# 2) CSV -> Partial->Remainder
# ==========================================
def build_partial_remainder(csv_path, partial_ratio=0.5):
    """
    Read CSV with columns like:
      CONVERSATION_ID, CONVERSATION_STEP, TEXT
    Group lines by conversation, sort by step, 
    build partial (first X% lines) vs remainder (last X% lines).
    Return list of (partial_str, remainder_str).
    """
    df = pd.read_csv(csv_path)
    pairs = []
    for convo_id, group in df.groupby("CONVERSATION_ID"):
        group_sorted = group.sort_values("CONVERSATION_STEP")
        texts = group_sorted["TEXT"].tolist()
        if len(texts) < 2:
            continue
        cutoff = max(1, int(len(texts)*partial_ratio))
        partial_list = texts[:cutoff]
        remainder_list= texts[cutoff:]
        partial_str = "\n".join(partial_list).strip()
        remainder_str= "\n".join(remainder_list).strip()
        if partial_str and remainder_str:
            pairs.append((partial_str, remainder_str))
    return pairs


In [4]:
# Example usage
csv_path = "C:/Users/DELL/Desktop/VOIP_Phishing_Attacks/Repos/convoPredict/conversation-prediction/FINAL_DATASET2.csv"  
partial_ratio = 0.5
pairs = build_partial_remainder(csv_path, partial_ratio)
print(f"Number of partial->remainder pairs: {len(pairs)}")
if pairs:
    print("Sample pair:\nPartial:", pairs[0][0], "\nRemainder:", pairs[0][1])


Number of partial->remainder pairs: 76
Sample pair:
Partial: Hello, this is [Your Name]'s personal assistant. How may I assist you today?
Hi, I'm Sam. I saw an ad about a photography workshop hosted by [Org Name] next month. I'm interested in registering but had a few questions.
Hi Sam, it's great to hear of your interest in the photography workshop. I'd be happy to help with any questions you have.
Thanks! I was wondering about the skill level required for participants. I'm fairly new to photography.
The workshop is designed to accommodate all skill levels, from beginners to more experienced photographers. [Org Name] aims to ensure everyone can learn and grow, regardless of their starting point.
That sounds perfect. What's the registration process? 
Remainder: You can register through our website. I can guide you through the steps if you'd like, or send you a direct link to the registration page.
A direct link would be great. Can you also tell me about the workshop fee?
Certainly, the

In [5]:
# ==========================================
# 3) Create a HuggingFace Datasets object
#    from the partial->remainder pairs
# ==========================================
from datasets import Dataset as HFDataset

def create_hf_dataset(pairs):
    # We'll build a list of dicts: { "source":..., "target":... }
    data_dict = {
        "source": [p[0] for p in pairs],
        "target": [p[1] for p in pairs]
    }
    hf_ds = HFDataset.from_dict(data_dict)
    return hf_ds

hf_ds = create_hf_dataset(pairs)
print(hf_ds)
if len(hf_ds)>0:
    print("Sample record:", hf_ds[0])


Dataset({
    features: ['source', 'target'],
    num_rows: 76
})
Sample record: {'source': "Hello, this is [Your Name]'s personal assistant. How may I assist you today?\nHi, I'm Sam. I saw an ad about a photography workshop hosted by [Org Name] next month. I'm interested in registering but had a few questions.\nHi Sam, it's great to hear of your interest in the photography workshop. I'd be happy to help with any questions you have.\nThanks! I was wondering about the skill level required for participants. I'm fairly new to photography.\nThe workshop is designed to accommodate all skill levels, from beginners to more experienced photographers. [Org Name] aims to ensure everyone can learn and grow, regardless of their starting point.\nThat sounds perfect. What's the registration process?", 'target': "You can register through our website. I can guide you through the steps if you'd like, or send you a direct link to the registration page.\nA direct link would be great. Can you also tell me

In [6]:
# We'll do a train/val split
train_size = int(0.9 * len(hf_ds))
val_size   = len(hf_ds) - train_size
hf_train, hf_val = hf_ds.train_test_split(test_size=val_size).values()

print("HF train size:", len(hf_train))
print("HF val size  :", len(hf_val))


HF train size: 68
HF val size  : 8


In [7]:
# ==========================================
# 4) Load BART + Tokenizer
# ==========================================
model_name = "facebook/bart-base"  # or bart-large, etc.
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
model.to(device)
print("Loaded BART + Tokenizer from:", model_name)


Loaded BART + Tokenizer from: facebook/bart-base


In [8]:
# ==========================================
# 5) Tokenization Function
# ==========================================
def tokenize_function(examples):
    # For BART seq2seq: 
    # "source" -> encoder input
    # "target" -> decoder labels
    model_inputs = tokenizer(
        examples["source"], 
        max_length=128,    # <--- param: tune
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target"], 
            max_length=128,  # <--- param: tune
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

hf_train = hf_train.map(tokenize_function, batched=True, remove_columns=["source","target"])
hf_val   = hf_val.map(tokenize_function,   batched=True, remove_columns=["source","target"])

hf_train.set_format("torch")
hf_val.set_format("torch")

print("Tokenized train sample:", hf_train[0])


Map:   0%|          | 0/68 [00:00<?, ? examples/s]c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 8/8 [00:00<00:00, 157.66 examples/s]

Tokenized train sample: {'input_ids': tensor([    0,   713,  2175,    16,   223,   110,   766,     6,    45,   223,
           84,   827,    18,   766,     4, 50118,  9904,     6,  4420,     4,
        50118, 32730,     6,    52,  1395,  1719,     5,   455,   379,   153,
        20858,   771,    19,    84,   827,    18,  1188,     4, 50118,  9904,
            6,  4420,     4, 50118, 13984,     6,   965,    75,    14,  4577,
          116, 50118,  9904,     6,  4420,     4, 50118,  1106,    84,   827,
        14617,    24,  4378,     6,    24,   115,    28, 32085,    25,    10,
        15178,  4628,  2541,     4, 50118,  9904,     4, 50118,   170,    40,
         1719,   843,    12,  1096,   207,    31,     5,  1049,  6084,     8,
          291,    12,   541,   207,    31,     5,   400,  6084,     4, 50118,
         9904,     4, 50118,   133,  2405,   291,    12,   541,   207,   782,
            7,    28,  2913,    30,    47,     6,     8,    52,   581, 27736,
           47,   423,     

In [9]:
# ==========================================
# 6) DataCollatorForSeq2Seq
# ==========================================
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    return_tensors="pt"
)


In [10]:
# ==========================================
# 7) Trainer Setup
# ==========================================
training_args = TrainingArguments(
    output_dir="bart_conversation_predict_2",  # <--- param: where to save
    overwrite_output_dir=True,
    num_train_epochs=3,                       # <--- param: tune
    per_device_train_batch_size=2,            # <--- param: tune
    per_device_eval_batch_size=2,
    learning_rate=5e-5,                       # <--- param: tune
    weight_decay=0.01,
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    logging_steps=10,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)


c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\DELL\AppData\Local\Temp\ipykernel_15632\3276508228.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
# ==========================================
# 8) Fine-Tune
# ==========================================
trainer.train()


  0%|          | 0/102 [00:00<?, ?it/s]c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\transformers\data\data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
 10%|▉         | 10/102 [00:48<05:13,  3.41s/it]

{'loss': 3.887, 'grad_norm': 9.756758689880371, 'learning_rate': 4.5098039215686275e-05, 'epoch': 0.29}


 20%|█▉        | 20/102 [01:20<04:18,  3.15s/it]

{'loss': 3.3955, 'grad_norm': 10.599475860595703, 'learning_rate': 4.0196078431372555e-05, 'epoch': 0.59}


 29%|██▉       | 30/102 [01:52<03:48,  3.17s/it]

...

                                                 
100%|██████████| 102/102 [06:00<00:00,  3.09s/it]

{'eval_loss': 2.4451236724853516, 'eval_runtime': 2.3735, 'eval_samples_per_second': 3.371, 'eval_steps_per_second': 1.685, 'epoch': 3.0}


100%|██████████| 102/102 [06:03<00:00,  3.56s/it]


{'train_runtime': 363.3148, 'train_samples_per_second': 0.561, 'train_steps_per_second': 0.281, 'train_loss': 2.756316970376407, 'epoch': 3.0}


TrainOutput(global_step=102, training_loss=2.756316970376407, metrics={'train_runtime': 363.3148, 'train_samples_per_second': 0.561, 'train_steps_per_second': 0.281, 'total_flos': 14915916288000.0, 'train_loss': 2.756316970376407, 'epoch': 3.0})

In [12]:
# ==========================================
# 9) Save Fine-Tuned Model
# ==========================================
save_dir = "bart_conversation_finetuned_model_2"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("Fine-tuned BART model + tokenizer saved to:", save_dir)


Fine-tuned BART model + tokenizer saved to: bart_conversation_finetuned_model_2


: 